In [1]:
!pip install -qqq ml-collections

In [1]:
import os
os.chdir("..")

import jax
import ml_collections

import pandas as pd

import glob
from datetime import datetime

import experiments.movielens_exp as movielens_run
import experiments.mnist_exp as mnist_run
import experiments.tabular_exp as tabular_run
import experiments.tabular_subspace_exp as tabular_sub_run

print(jax.device_count())

INFO:absl:Unable to initialize backend 'tpu_driver': NOT_FOUND: Unable to find driver in registry given worker: 
INFO:absl:Unable to initialize backend 'gpu': NOT_FOUND: Could not find registered platform with name: "cuda". Available platform names are: Host Interpreter TPU


8


In [2]:
def get_config(filepath):
  """Get the default hyperparameter configuration."""
  config = ml_collections.ConfigDict()
  config.filepath = filepath
  config.ntrials = 10
  return config

In [3]:
timestamp = datetime.timestamp(datetime.now())

# Run tabular experiments

In [ ]:
tabular_filename = f"./results/tabular_results_{timestamp}.csv"
config = get_config(tabular_filename)
tabular_run.main(config)

Environment :  shuttle
	Bandit : Linear
		Expected Reward : 4426.40 ± 9.98
		Time : 8.301s
	Bandit : Linear KF
		Expected Reward : 4426.50 ± 12.08
		Time : 6.001s
	Bandit : Linear Wide
		Expected Reward : 4208.80 ± 7.95
		Time : 11.835s
	Bandit : Limited Neural Linear


# Run MNIST experiments

In [ ]:
mnist_filename = f"./results/mnist_results_{timestamp}.csv"
config = get_config(mnist_filename)
mnist_run.main(config)

# Run movielens experiments

In [ ]:
movielens_filename = f"./results/movielens_results_{timestamp}.csv"
config = get_config(movielens_filename)
movielens_run.main(config)

# Run tabular subspace experiment

In [ ]:
tabular_sub_filename = f"./results/tabular_subspace_results_{datetime.timestamp(datetime.now())}.csv"
config = get_config(tabular_sub_filename)
tabular_sub_run.main(config)